In [1]:
import matplotlib
import numpy as np
import os
import torch
import cv2

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
from torch.autograd import Variable

In [3]:
%cd SuperGluePretrainedNetwork/

/home/ichuviliaeva/ocr_course/course_ocr/task1/SuperGluePretrainedNetwork


In [4]:
from models.superpoint import SuperPoint
from models.utils import read_image

In [5]:
from models.superglue import SuperGlue

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
gpu = torch.device('cuda:0')

In [7]:
DATASET_PATH = Path('/DATA/ichuviliaeva/midv500_compressed/')
assert DATASET_PATH.exists(), DATASET_PATH.absolute()
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

gt_root =  /DATA/ichuviliaeva/midv500_compressed/12_deu_drvlic_new/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/12_deu_drvlic_new/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/33_mac_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/33_mac_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/50_xpo_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/50_xpo_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/38_rou_drvlic/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/38_rou_drvlic/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/27_hrv_passport/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/27_hrv_passport/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/44_ukr_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/44_ukr_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/23_fin_drvlic/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/23_f

(50, course_ocr_t1.data.MidvPackage)

In [8]:
spoint = SuperPoint({'keypoint_threshold': 0.05}).to(gpu)
sglue = SuperGlue(config={'match_threshold': 0.5, 'sinkhorn_iterations': 1}).to(gpu)

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [14]:
class baseline_glu:
    def __init__(self, template_item):
        self.template_item = template_item
        
        self.template_img, self.template_img_t, _ = read_image(
            self.template_item.img_path,
            device=gpu,
            resize=(int(self.template_item.image_size[0]), int(self.template_item.image_size[1])), 
            rotation=0, 
            resize_float=True
        )
        
        self.template_kps = spoint({'image': self.template_img_t})
        
        for k in self.template_kps:
            if type(self.template_kps[k]) == torch.Tensor:
                self.template_kps[k] = Variable(self.template_kps[k].to(gpu)) # , requires_grad=True)
            else:
                self.template_kps[k] = Variable(torch.stack(self.template_kps[k]).to(gpu)) # , requires_grad=True)
            
    def run_superglue(self, item_kps, item_img_t):
        sg_input = dict()
        
        for (k, v) in self.template_kps.items():
            sg_input[k + '0'] = v
            sg_input['image0'] = self.template_img_t
            
        for (k, v) in item_kps.items():
            sg_input[k + '1'] = v
            sg_input['image1'] = item_img_t
        
        sg_output = sglue(sg_input)
        idx0 = sg_output['matches0'] > -1
        template_kps_cleared = sg_input['keypoints0'][0][idx0[0]]
        idx1 = sg_output['matches0'][idx0]
        item_kps_cleared = sg_input['keypoints1'][0][idx1]
        
        return template_kps_cleared, item_kps_cleared
    
    def compute_crop(self, H):
        w, h = self.template_item.image_size
        h = float(h)
        w = float(w)
        corners = [[0, 0], [w, 0], [w, h], [0, h]]
        corners_cropped = []
        
        for c in corners:
            vec = H @ np.array([c[0], c[1], 1])
            corners_cropped.append([vec[0] / vec[-1], vec[1] / vec[-1]])
        return np.array(corners_cropped) / np.array([w, h])
    
    def make_matches(self, item):
        item_img, item_img_t, _ = read_image(
            item.img_path,
            device=gpu,
            resize=(int(item.image_size[0]), int(item.image_size[1])), 
            rotation=0, 
            resize_float=True
        )
        
        item_kps = spoint({'image': item_img_t})
        for k in item_kps:
            if type(item_kps[k]) == torch.Tensor:
                item_kps[k] = Variable(item_kps[k].to(gpu))
            else:
                item_kps[k] = Variable(torch.stack(item_kps[k]).to(gpu))
            
        template_kps_cleared, item_kps_cleared = self.run_superglue(item_kps, item_img_t)
        return template_kps_cleared, item_kps_cleared
    
    def predict(self, item):
        template_kps_cleared, item_kps_cleared = self.make_matches(item)
        (H, mask) = cv2.findHomography(np.array(template_kps_cleared.detach().cpu()),
                                       np.array(item_kps_cleared.detach().cpu()), 
                                       method=cv2.RANSAC)
        return self.compute_crop(H)

In [19]:
def count_loss(template_item, test_item, matches_template, matches_item):
    w, h = template_item.image_size
    t_w, t_h = test_item.image_size
    h = float(h)
    w = float(w)
    t_h = float(t_h)
    t_w = float(t_w)
    corners = [[0, 0], [w, 0], [w, h], [0, h]]
    
    loss = Variable(torch.tensor(0).float().to(gpu), requires_grad = True)
    # можно предпосчитать, пока непонятно, насколько это ускорит
    (H_ideal, mask) = cv2.findHomography(
        np.array(corners),
        np.array(test_item.gt_data['quad']),
        method=cv2.RANSAC)
    H_ideal = Variable(torch.tensor(H_ideal).float().to(gpu))   #, requires_grad=True)
    
    for i, p in enumerate(matches_template):
        vec = H_ideal @ torch.stack([p[0], p[1], torch.tensor(1.0).to(gpu)])
        loss = loss + torch.sqrt(
            (vec[0] / vec[-1] - matches_item[i][0])**2 / t_w**2 + \
            (vec[1] / vec[-1] - matches_item[i][1])**2 / t_h**2
        )
        
    return loss / len(matches_template)

In [20]:
def train(epochs, optimizer, lr):
    
    for t in tqdm(range(epochs)):
        sglue.train()
        if t == 0:
            optimizer.lr = lr / 1000
        else:
            optimizer.lr = lr
            
        optimizer.zero_grad()
        
        loss = 0
        for dp in tqdm(data_packs):
            # закинуть sglu в baseline вот тут
            # или даже раскатать baseline внутри train
            glu = baseline_glu(dp.template_item)
            for i in range(len(dp)):
                if dp[i].is_test_split() == False:
                    matches_template, matches_item = glu.make_matches(dp[i])
                    loss += count_loss(dp.template_item, dp[i], matches_template, matches_item)
        
        loss.backward()
        optimizer.step()
        print('epoch ', t, ': loss = ', loss)

In [21]:
optimizer = torch.optim.Adam(sglue.parameters(), lr = 0.001)

In [22]:
train(3, optimizer, lr = 0.001)

  0%|                                                                        | 0/50 [00:00<?, ?it/s]


  2%|█▎                                                              | 1/50 [00:38<31:30, 38.57s/it]


 22%|█████████████▊                                                 | 11/50 [03:13<10:06, 15.54s/it]


 33%|█████████████████████▎                                          | 1/3 [11:51<23:43, 711.52s/it]

epoch  0 : loss =  tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)


...


  0%|                                                                        | 0/50 [00:00<?, ?it/s]


  2%|█▎                                                              | 1/50 [00:39<31:57, 39.12s/it]


 22%|█████████████▊                                                 | 11/50 [03:13<10:00, 15.41s/it]


100%|████████████████████████████████████████████████████████████████| 3/3 [35:40<00:00, 713.40s/it]

epoch  2 : loss =  tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)


In [23]:
results_dict = dict()

for dp in tqdm(data_packs):
    glu = baseline_glu(dp.template_item)
    for i in range(len(dp)):
        if dp[i].is_test_split():
            try:
                results_dict[dp[i].unique_key] = glu.predict(dp[i])
            except Exception as exc:
                # Для пропущенных в словаре ключей в метриках автоаматически засчитается IoU=0
                print(exc)

  0%|                                                                        | 0/50 [00:00<?, ?it/s]

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

  2%|█▎                                                              | 1/50 [00:08<06:35,  8.07s/it]

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

...

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/ope

100%|███████████████████████████████████████████████████████████████| 50/50 [04:28<00:00,  5.38s/it]

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'

OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'



In [24]:
from course_ocr_t1.metrics import dump_results_dict, measure_crop_accuracy

In [26]:
%cd ..

/home/ichuviliaeva/ocr_course/course_ocr/task1


In [27]:
dump_results_dict(results_dict, Path() / 'pred.json')

In [28]:
acc = measure_crop_accuracy(
    Path() / 'pred.json',
    Path() / 'gt.json'
)

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.inte

In [29]:
print("Точность кропа: {:1.4f}".format(acc))

Точность кропа: 0.0000


Надо всё же сделать предподсчёты, чтобы всё на свете не выбрасывать

In [30]:
kp_templates = []
image_t_templates = []
kp_items = []
image_t_items = []
Hs = []

In [32]:
for dp in tqdm(data_packs):
    template_img, template_img_t, _ = read_image(
        dp.template_item.img_path,
        device=gpu,
        resize=(int(dp.template_item.image_size[0]), int(dp.template_item.image_size[1])), 
        rotation=0, 
        resize_float=True
    )
        
    kp_templates.append(spoint({'image': template_img_t}))
    image_t_templates.append(template_img_t)
    
    w, h = dp.template_item.image_size
    h = float(h)
    w = float(w)
    corners = [[0, 0], [w, 0], [w, h], [0, h]]
    
    kp_small = []
    H_small = []
    image_t_small = []
    for i in range(len(dp)):
        item_img, item_img_t, _ = read_image(
            dp[i].img_path,
            device=gpu,
            resize=(int(dp[i].image_size[0]), int(dp[i].image_size[1])), 
            rotation=0, 
            resize_float=True
        )
        kp_small.append(spoint({'image': item_img_t}))
        image_t_small.append(item_img_t)
        
        (H_ideal, mask) = cv2.findHomography(
        np.array(corners),
        np.array(dp[i].gt_data['quad']),
        method=cv2.RANSAC)
        H_small.append(H_ideal)
        
    Hs.append(h_small)         
    kp_items.append(kp_small)
    image_t_items.append(image_t_small)

  0%|                                                                        | 0/50 [00:00<?, ?it/s]

  0%|                                                                        | 0/50 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 88.00 MiB (GPU 0; 23.65 GiB total capacity; 20.67 GiB already allocated; 43.56 MiB free; 20.75 GiB reserved in total by PyTorch)

In [ ]:
def train(epochs, optimizer, lr):
    
    for t in tqdm(range(epochs)):
        sglue.train()
        if t == 0:
            optimizer.lr = lr / 1000
        else:
            optimizer.lr = lr
            
        optimizer.zero_grad()
        
        loss = 0
        for i, dp in tqdm(enumerate(data_packs)):
            template_kps = kp_templates[i]
            template_image_t = image_t_templates[i]
            
            for k in template_kps:
                if type(template_kps[k]) == torch.Tensor:
                    template_kps[k] = Variable(template_kps[k].to(gpu))
                else:
                    template_kps[k] = Variable(torch.stack(template_kps[k]).to(gpu))
                    
            for j in range(len(dp)):
                if dp[j].is_test_split() == False:
                    item_kps = kp_items[i][j]
                    item_image_t = image_t_items[i][j]
                    for k in item_kps:
                        if type(item_kps[k]) == torch.Tensor:
                            item_kps[k] = Variable(item_kps[k].to(gpu))
                        else:
                            item_kps[k] = Variable(torch.stack(item_kps[k]).to(gpu))
            
                    sg_input = dict()
                    for (k, v) in template_kps.items():
                        sg_input[k + '0'] = v
                        sg_input['image0'] = template_img_t
                        
                    for (k, v) in item_kps.items():
                        sg_input[k + '1'] = v
                        sg_input['image1'] = item_img_t
                        
                    sg_output = sglue(sg_input)
                    idx0 = sg_output['matches0'] > -1
                    template_kps_cleared = sg_input['keypoints0'][0][idx0[0]]
                    idx1 = sg_output['matches0'][idx0]
                    item_kps_cleared = sg_input['keypoints1'][0][idx1]
                    
                    loss = loss + count_loss(dp.template_item, dp[j], 
                                             template_kps_cleared, item_kps_cleared)
        loss.backward()
        optimizer.step()
        print('epoch ', t, ': loss = ', loss)

In [ ]:
def eval(refer_item_num, refer_item, test_item_num):
    sglue.eval()
    template_kps = kp_templates[refer_item_num]
    template_image_t = image_t_templates[refer_item_num]
            
    for k in template_kps:
        if type(template_kps[k]) == torch.Tensor:
            template_kps[k] = Variable(template_kps[k].to(gpu))
        else:
            template_kps[k] = Variable(torch.stack(template_kps[k]).to(gpu))
                    
    item_kps = kp_items[refer_item_num][test_item_num]
    item_image_t = image_t_items[refer_item_num][test_item_num]
    for k in item_kps:
        if type(item_kps[k]) == torch.Tensor:
            item_kps[k] = Variable(item_kps[k].to(gpu))
        else:
            item_kps[k] = Variable(torch.stack(item_kps[k]).to(gpu))
            
    sg_input = dict()
    for (k, v) in template_kps.items():
        sg_input[k + '0'] = v
        sg_input['image0'] = template_img_t
                        
    for (k, v) in item_kps.items():
        sg_input[k + '1'] = v
        sg_input['image1'] = item_img_t
                        
    sg_output = sglue(sg_input)
    idx0 = sg_output['matches0'] > -1
    template_kps_cleared = sg_input['keypoints0'][0][idx0[0]]
    idx1 = sg_output['matches0'][idx0]
    item_kps_cleared = sg_input['keypoints1'][0][idx1]
     
    (H, mask) = cv2.findHomography(np.array(template_kps_cleared.detach().cpu()),
                                   np.array(item_kps_cleared.detach().cpu()), 
                                   method=cv2.RANSAC)    
    w, h = refer_item.image_size
    h = float(h)
    w = float(w)
    corners = [[0, 0], [w, 0], [w, h], [0, h]]
    corners_cropped = []
        
    for c in corners:
        vec = H @ np.array([c[0], c[1], 1])
        corners_cropped.append([vec[0] / vec[-1], vec[1] / vec[-1]])
    return np.array(corners_cropped) / np.array([w, h])